In [2]:
## 整理peaks数量数据, 20230512
import pandas as pd
from functools import reduce
df_rep1 = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/J668_glr48_cutTag_Rep1.bed", sep=" ", names=['rep1', 'a', 'material', 'strain', 'time', 'chip_tag'])
df_rep2 = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/J668_glr48_cutTag_Rep2.bed", sep=" ", names=['rep2', 'a', 'material', 'strain', 'time', 'chip_tag'])
df_merge = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/J668_glr48_cutTag_merge.bed", sep=" ", names=['merge', 'a', 'material', 'strain', 'time', 'chip_tag'])
df_IDR = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/J668_glr48_cutTag_IDR.bed", sep=" ", names=['IDR','material', 'strain', 'time', 'chip_tag'])
df_rep12_merge = reduce(lambda left,right: pd.merge(left, right, on=['a','material', 'strain', 'time', 'chip_tag']), [df_rep1,df_rep2,df_merge])
df_rep12_merge.drop("a", axis=1, inplace=True)
df_result = pd.merge(df_rep12_merge, df_IDR, how='inner', on=['material', 'strain', 'time', 'chip_tag'])
df_result['ratio'] = (df_result['IDR']*2)/(df_result['rep1'] + df_result['rep2'])


# add FRiP
df_FRiP = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/FRiP.bed", sep=" ", names=['material', 'strain', 'time', 'rep', 'chip_tag', 'number'])
df_rep1_FRiP = df_FRiP.query("rep == 'Rep1'")[['material', 'strain', 'time', 'chip_tag', 'number']].copy()
df_rep2_FRiP = df_FRiP.query("rep == 'Rep2'")[['material', 'strain', 'time', 'chip_tag', 'number']].copy()
df_merge_FRiP = df_FRiP.query("rep == 'merge'")[['material', 'strain', 'time', 'chip_tag', 'number']].copy()
df_rep1_FRiP.rename(columns={'number':'rep1_FRiP'}, inplace=True)
df_rep2_FRiP.rename(columns={'number':'rep2_FRiP'}, inplace=True)
df_merge_FRiP.rename(columns={'number':'merge_FRiP'}, inplace=True)
df_result = reduce(lambda left,right: pd.merge(left, right, on=['material', 'strain', 'time', 'chip_tag']), [df_result, df_rep1_FRiP, df_rep2_FRiP, df_merge_FRiP])

## add recall peaks
df_recall_peaks = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/J668_glr48_recalled_peaks.bed", sep=" ", names=['recalled','material', 'strain', 'time','chip_tag'])
df_result = pd.merge(df_result, df_recall_peaks, how='inner', on=['material', 'strain', 'time','chip_tag'])

df_result.to_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/peaks_number/J668_glr48_peaks_data.csv", index=False)

In [4]:
## statistic number of diffpeaks
## Nutures
## 20230522
import pandas as pd

# 1、 calculate the number of diffpeaks between
def diffpeaks_number(df):
    df_up = df.query("logfc>0").copy()
    df_down = df.query("logfc<0").copy()
    up_number, down_number = df_up.shape[0], df_down.shape[0]
    return up_number, down_number

# 2、different strain or sample or time
def diff_strain_sample_time(dir):
    Times = ['0min', '5min', '10min', '30min', '180min', '720min']
    df_return = pd.DataFrame(columns = ['sample1', 'sample2', 'sample3', 'chip', 'times', 'methods', 'number', 'tags'])
    # 1、diff strain or sample
    for s in ['J668_Fov7_V991', 'J668_Glr19_Fov7']:
        if s == 'J668_Fov7_V991':
            sample1, sample2 = 'J668_Fov7', 'J668_V991'
        else:
            sample1, sample2 = 'J668_Fov7', 'Glr19_Fov7'
        for k in ['H3K27ac', 'H3K27me3']:
            for t in Times:
                for h in ['deseq2', 'edgeR']:
                    input_f = "{0}/{1}/{2}/{3}_{4}-{5}_{4}/{3}_{4}_{5}_{4}_{6}_sig.bed".format(dir,s,k,sample1, t, sample2, h)
                    df = pd.read_csv(input_f, sep="\t", names=['chrom', 'start', 'end', 'a', 'logfc'])
                    up_number, down_number = diffpeaks_number(df)
                    df_return.loc[df_return.shape[0],  ] = [sample1, sample2,"{0}-{1}".format(sample1, sample2), k, t, h, up_number, 'Up']
                    df_return.loc[df_return.shape[0],  ] = [sample1, sample2, "{0}-{1}".format(sample1, sample2), k, t, h, down_number, 'Down']
    # 2、diff time
    df_return2 = pd.DataFrame(columns = ['samples', 'chip', 'time1', 'time2', 'times', 'methods', 'number', 'tags'])
    for s in ['J668_Fov7', 'J668_V991', 'Glr19_Fov7']:
        for k in ['H3K27ac', 'H3K27me3']:
            for i in range(5):
                times1 = Times[i]
                times2 = Times[i+1]
                for h in ['deseq2', 'edgeR']:
                    input_f = "{0}/{1}/{2}/{1}_{3}-{1}_{4}/{1}_{3}_{1}_{4}_{5}_sig.bed".format(dir, s, k, times1, times2, h)
                    df = pd.read_csv(input_f, sep="\t", names=['chrom', 'start', 'end', 'a', 'logfc'])
                    up_number, down_number = diffpeaks_number(df)
                    df_return2.loc[df_return2.shape[0],  ] = [s, k, times1, times2, "{0}-{1}".format(times1, times2), h, up_number, 'Up']
                    df_return2.loc[df_return2.shape[0],  ] = [s, k, times1, times2, "{0}-{1}".format(times1, times2), h, down_number, 'Down']
    return df_return, df_return2

df_diff_strain, df_diff_time = diff_strain_sample_time("/public/home/xhhuang/biodate/VD_FOV_3Dgenome/CUT_Tag/diff_peaks")
df_diff_strain.to_csv("J668_diff_strain.csv", index=False)
df_diff_time.to_csv("J668_diff_time.csv", index=False)

In [8]:
## 分析注释在基因区近端的调控元件数目
import pandas as pd
df = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/diffpeaks/J668_V991/H3K27me3/J668_V991_0min_J668_V991_5min_peaks_annotation.bed", sep=" ")
df_proximal = df.query("annotation!='Distal Intergenic'").copy()
#df_proximal.drop_duplicates(keep='first', subset=['geneId'], inplace=True)
df_proximal.to_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/diffpeaks/J668_V991/H3K27me3/J668_V991_0min_5min_proximal_geneid.csv", index=False, columns=['geneId'])

In [9]:
## 分析注释在基因区近端的调控元件数目
import pandas as pd
df = pd.read_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/diffpeaks/J668_V991/H3K27ac/J668_V991_0min_J668_V991_5min_peaks_annotation.bed", sep=" ")
df_proximal = df.query("annotation!='Distal Intergenic'").copy()
#df_proximal.drop_duplicates(keep='first', subset=['geneId'], inplace=True)
df_proximal.to_csv("G:/Billfish/VD_FOV_3Dgenome/Cut_tag/rawdata/diffpeaks/J668_V991/H3K27ac/J668_V991_0min_5min_proximal_geneid.csv", index=False, columns=['geneId'])